In [ ]:
!gdown --id 1KIRvJyDcLVsKm206Ge8A6pqylg1NA65S
!unzip /content/QALB-0.9.0-May04-2021-SharedTask.zip

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
def read_data(Lines):
  newline = [i for i in range(len(Lines)) if Lines[i] == '\n' ]
  sen =[]
  all = []
  for i in range(len(newline)):
    if i == 0:
      sen = Lines[0:newline[i]]
      all.append(sen)
    else:
      sen = Lines[newline[i-1]+1:newline[i]]
      all.append(sen)
  return all

In [ ]:
def parse_correction(correction_line):
  correction_line = correction_line[2:].split('|||') 
  start_id, end_id = map(int, correction_line[0].split())
  return (start_id, end_id, correction_line[1], correction_line[2])

In [ ]:
def parse_data(data):
  all_edited = []
  for lst in data:
    for i in range(len(lst)):
      if i == 0:
        lst[i] = lst[i].split('S')[1].split('\n')[0]
      else:
        lst[i]  = parse_correction(lst[i])
    all_edited.append(lst)
  return all_edited

In [ ]:
def apply_corrections(text, corrections):
  words = text.split()
  for start, end, _, content in reversed(corrections):
    words = words[:start] + content.split() + words[end:]
  return ' '.join(words)

In [ ]:
train_df = pd.DataFrame({
    'original': [],
    'corrected': []
}) 
test_df = pd.DataFrame({
    'original': [],
    'corrected': []
}) 
dev_df = pd.DataFrame({
    'original': [],
    'corrected': []
}) 

In [ ]:
test_m2 = open('/content/QALB-0.9.0-May04-2021-SharedTask/data/QALB-Train2014.m2')
m = test_m2.readlines()

In [ ]:
all = read_data(m)

In [ ]:
all[0]

['S الى التعليق رقم 2 اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور . فليتهم يفعلونها ولو مرة . ولنا نحن كشعوب مسؤولية كذالك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينينة وليتنا نفعلها ولو ثلاتة اشهر .\n',
 'A 0 1|||Edit|||إلى|||REQUIRED|||-NONE-|||0\n',
 'A 4 4|||Add_before|||:|||REQUIRED|||-NONE-|||0\n',
 'A 4 5|||Edit|||أكيد|||REQUIRED|||-NONE-|||0\n',
 'A 5 6|||Edit|||أن|||REQUIRED|||-NONE-|||0\n',
 'A 6 7|||Edit|||للحكام|||REQUIRED|||-NONE-|||0\n',
 'A 11 12|||Edit|||أدناها|||REQUIRED|||-NONE-|||0\n',
 'A 23 24|||Edit|||!|||REQUIRED|||-NONE-|||0\n',
 'A 28 29|||Edit|||كذلك|||REQUIRED|||-NONE-|||0\n',
 'A 32 33|||Edit|||إخواننا|||REQUIRED|||-NONE-|||0\n',
 'A 37 38|||Edit|||الصينية|||REQUIRED|||-NONE-|||0\n',
 'A 38 38|||Add_before|||.|||REQUIRED|||-NONE-|||0\n',
 'A 41 42|||Edit|||ثلاثة|||REQUIRED|||-NONE-|||0\n',
 'A 42 43|||Edit|||أشهر|||REQUIRED|||-NONE-|||0\n',
 'A 43 44|||Edit|||!|||REQUIRED|||-NONE-|||0\n']

In [ ]:
all_edited = parse_data(all)
all_edited[0]

[' الى التعليق رقم 2 اكيد ان لحكام العرب والمسلمين مسؤولية يتمثل ادناها في استدعاء السفراء في الصين للتشاور . فليتهم يفعلونها ولو مرة . ولنا نحن كشعوب مسؤولية كذالك تتمثل في مساندة اخواننا في الصين بمقاطعة البضائع الصينينة وليتنا نفعلها ولو ثلاتة اشهر .',
 (0, 1, 'Edit', 'إلى'),
 (4, 4, 'Add_before', ':'),
 (4, 5, 'Edit', 'أكيد'),
 (5, 6, 'Edit', 'أن'),
 (6, 7, 'Edit', 'للحكام'),
 (11, 12, 'Edit', 'أدناها'),
 (23, 24, 'Edit', '!'),
 (28, 29, 'Edit', 'كذلك'),
 (32, 33, 'Edit', 'إخواننا'),
 (37, 38, 'Edit', 'الصينية'),
 (38, 38, 'Add_before', '.'),
 (41, 42, 'Edit', 'ثلاثة'),
 (42, 43, 'Edit', 'أشهر'),
 (43, 44, 'Edit', '!')]

In [ ]:
for i in range(len(all_edited)):
  train_df = train_df.append({'original': all_edited[i][0],
                              'corrected': apply_corrections(all_edited[i][0],all_edited[i][1:])}, ignore_index=True)

In [ ]:
train_df

,original,corrected
0,الى التعليق رقم 2 اكيد ان لحكام العرب والمسلم...,إلى التعليق رقم 2 : أكيد أن للحكام العرب والمس...
1,نحن ببالغ الاسى نعزي ضحايا الحادث الاليم الذي...,نحن ببالغ الأسى نعزي ضحايا الحادث الأليم الذي ...
2,للاسف ان الدم المهدور عراقي لكن بشكل عام لماذ...,للأسف إن الدم المهدور عراقي ، لكن بشكل عام لما...
3,واصل الثوار أنتصاراتهم وحررو كل حقول النفط ال...,واصل الثوار انتصاراتهم وحرروا كل حقول النفط ال...
4,يا حرام على ما يدعو نفسهم شبيحة الأسد وحزب ال...,يا حرام على من يدعون نفسهم شبيحة الأسد وحزب ال...
...,...,...
19406,ياعالم اطفالناونسائنا وشبابنا يقتلون ليلا ونه...,يا عالم أطفالنا ونساؤنا وشبابنا يقتلون ليلا ون...
19407,بسم الله الرحمن الرحيم : قالوا للمرى ( أمرأة ...,بسم الله الرحمن الرحيم . قالوا للمرى ( امرأة )...
19408,الله الله علي الظالمين ومن والاهم وهنيئا للمج...,الله الله على الظالمين ومن والاهم ، وهنيئا للم...
19409,ان قيادة فتح لـ م ت ف وقيادة هذه للشعب لسنوات...,إن قيادة فتح لم تف . وقيادة هذه للشعب لسنوات ه...


In [ ]:
test_m2 = open('/content/QALB-0.9.0-May04-2021-SharedTask/data/QALB-Test2014.m2')
m = test_m2.readlines()
all = read_data(m)
all_edited = parse_data(all)
for i in range(len(all_edited)):
  test_df = test_df.append({'original': all_edited[i][0],
                              'corrected': apply_corrections(all_edited[i][0],all_edited[i][1:])}, ignore_index=True)

In [ ]:
test_df

,original,corrected
0,لا زال كبير الشبيحه يظن ان ارواح وآلام الناس ...,لازال كبير الشبيحة يظن أن أرواح وآلام الناس أق...
1,تراهنون يا امريكان و يا اذنابهم على مارس طيب ...,تراهنون يا أمريكان ويا أذنابهم على مارس ، طيب ...
2,يشاع إعتقاد في المجال الجنائي أن اللص لا يميل...,يشاع اعتقاد في المجال الجنائي أن اللص لا يميل ...
3,تعجبني المظاهرات في حمص هنا وهناك مثل ما راين...,تعجبني المظاهرات في حمص هنا وهناك مثل ما رأينا...
4,الى مشاركه 56 انت تتكلم بالعكس الاحرار هم الذ...,إلى مشاركة 56 ، أنت تتكلم بالعكس ، الأحرار هم ...
...,...,...
963,رحم الله شهداءنا الابرار ايها الارهابي الحقير...,رحم الله شهداءنا الأبرار . أيها الإرهابيُّ الح...
964,رجاءلبعض الاخوة المشاركين : لا تفسروا الايات ...,رجاء لبعض الإخوة المشاركين : لا تفسروا الآيات ...
965,هذا السلاح لن يفيد إلا شركات السلاح الأمريكيه...,هذا السلاح لن يفيد إلا شركات السلاح الأمريكية ...
966,انا اعرف نتيجة الانتخابات من البدايه لازم يفو...,أنا أعرف نتيجة الانتخابات من البداية ، لازم يف...


In [ ]:
test_m2 = open('/content/QALB-0.9.0-May04-2021-SharedTask/data/QALB-Dev2014.m2')
m = test_m2.readlines()
all = read_data(m)
all_edited = parse_data(all)
for i in range(len(all_edited)):
  dev_df = dev_df.append({'original': all_edited[i][0],
                              'corrected': apply_corrections(all_edited[i][0],all_edited[i][1:])}, ignore_index=True)

In [ ]:
dev_df

,original,corrected
0,سبحان الله الحكام العرب سيموت على الكرسي ليضه...,سبحان الله ، الحكام العرب سيموت على الكرسي ليظ...
1,النصر آت لا محال انشاء الله من يؤمن بالله وال...,النصر آت لا محال إن شاء الله . من يؤمن بالله و...
2,الى كل شخص يعتقد ان بشار الاسد سوف يخرج من سو...,إلى كل شخص يعتقد أن بشار الأسد سوف يخرج من سور...
3,الأسد وعصابته لحد الآن غير مستوعبين إن و رح ي...,الأسد وعصابته - لحد الآن - غير مستوعبين أنه رح...
4,النظام الأسدي تجاوز حتى ما فعله معمر القذافي ...,النظام الأسدي تجاوز حتى ما فعله معمر القذافي ف...
...,...,...
1012,مالي فهمانة ليش ماحدى عبيحكي عن شهداء الجيش ا...,ماني فهمانة ليش ما أحد يحكي عن شهداء الجيش الس...
1013,اذكر كلامه جيدا عندما قال الشاب : ( جايينك لع...,أذكر كلامه جيدا ، عندما قال الشاب : { جايينك ل...
1014,لقد فشلت أمريكا وإسرائيل وفرنسا وحلفائهم من ا...,لقد فشلت أمريكا ، وإسرائيل ، وفرنسا ، وحلفائهم...
1015,الله معنا ولن يترنا اعمالنا وحماس حركة ربانية...,الله معنا ، ولن يتر أعمالنا . وحماس حركة رباني...


In [ ]:
train_df['corrected'][1]

'نحن ببالغ الأسى نعزي ضحايا الحادث الأليم الذي ذهب ضحيته حوالي 77 ما بين جريح ومتوفي . وأطالب من سلطات بلادي أن تطبق القانون بصرامة ، وتضرب بيد من حديد مثل هؤلاء المجرمين ، والمتهورين ، والمتسببين في مثل هذه الحوادث . رحم الله الضحايا وأسكنهم فسيح جنانه ، وألهم ذويهم الصبر والسلوان . إنا لله وإنا إليه راجعون .'

In [ ]:
class preprocess:

  def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    #text = re.sub("ؤ", "ء", text)
    #text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("(\d+)([\u0621-\u063A\u0641-\u064A\u0660-\u066C]+)", r" \1 \2 ", text)
    text = re.sub("([\u0621-\u063A\u0641-\u064A\u0660-\u066C]+)(\d+)", r" \1 \2 ", text)
    return text

  def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)
    return text


  def remove_punctuations(text):
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    english_punctuations = string.punctuation
    punctuations_list = arabic_punctuations + english_punctuations
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

  def remove_repeating_char(text):
    return re.sub(r'(..)\1+', r'\1', text)

In [ ]:
def pre(text):
  text = preprocess.normalize_arabic(text)
  text = preprocess.remove_repeating_char(text)
  text = preprocess.remove_punctuations(text)
  text = preprocess.remove_diacritics(text)
  return text

In [ ]:
train_df.corrected = train_df.corrected.apply(pre)
train_df.original = train_df.original.apply(pre)

In [ ]:
train_df

,original,corrected
0,الي التعليق رقم 2 اكيد ان لحكام العرب والمسلم...,الي التعليق رقم 2 اكيد ان للحكام العرب والمسل...
1,نحن ببالغ الاسي نعزي ضحايا الحادث اليم الذي ذ...,نحن ببالغ الاسي نعزي ضحايا الحادث اليم الذي ذه...
2,للاسف ان الدم المهدور عراقي لكن بشكل عام لماذ...,للاسف ان الدم المهدور عراقي لكن بشكل عام لماذ...
3,واصل الثوار انتصاراتهم وحررو كل حقول النفط ال...,واصل الثوار انتصاراتهم وحرروا كل حقول النفط ال...
4,يا حرام علي ما يدعو نفسهم شبيحه الاسد وحزب ال...,يا حرام علي من يدعون نفسهم شبيحه الاسد وحزب ال...
...,...,...
19406,ياعالم اطفالناونسائنا وشبابنا يقتلون ليلا ونه...,يا عالم اطفالنا ونساؤنا وشبابنا يقتلون ليلا ون...
19407,بسم الله الرحمن الرحيم قالوا للمري امراه غ...,بسم الله الرحمن الرحيم قالوا للمري امراه غط...
19408,الله الله علي الظالمين ومن والاهم وهنيئا للمج...,الله الله علي الظالمين ومن والاهم وهنيئا للمج...
19409,ان قياده فتح ل م ت ف وقياده هذه للشعب لسنوات ...,ان قياده فتح لم تف وقياده هذه للشعب لسنوات هي...


In [ ]:
with open('train_data_correct.txt','w') as f:
  for i in range(len(train_df)):
    f.write(train_df['corrected'][i]+'\n')

In [ ]:
with open('train_data_wrong.txt','w') as f:
  for i in range(len(train_df)):
    f.write(train_df['original'][i]+'\n')